Model implementation from
http://nlp.seas.harvard.edu/annotated-transformer/#training

In [ ]:
import tensorflow as tf

In [ ]:
!pip install -q torchdata==0.3.0 spacy==3.2 altair GPUtil
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.1 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.9 MB 4.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
!pip install -U torchtext==0.12.0
!pip install torch==1.11.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

In [ ]:
def is_interactive_notebook():
  return __name__ == "__main__"

def show_example(fn, args=[]):
  if __name__ == "__main__" and RUN_EXAMPLES:
    return fn(*args)

class DummyOptimizer(torch.optim.Optimizer):
  def __init__(self):
    self.param_groups = [{"lr":0}]
    None

  def step(self):
    None
  
  def zero_grad(self, set_to_none=False):
    None
  

class DummyScheduler:
  def step(self):
    None

In [ ]:
class EncoderDecoder(nn.Module):
  def __init__(self,encoder,decoder,src_embed,tgt_embed,generator):
    super(EncoderDecoder, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.generator = generator

  def forward(self, src, tgt, src_mask, tgt_mask):
    return self.decode(self.encode(src,src_mask),src_mask,tgt,tgt_mask)
  
  def encode(self, src, src_mask):
    return self.encoder(self.src_embed(src),src_mask)
  
  def decode(self, memory, src_mask, tgt, tgt_mask):
    return self.decoder(self.tgt_embed(tgt),memory,src_mask,tgt_mask)

In [ ]:
class Generator(nn.Module):
  def __init__(self, d_model, vocab):
    super(Generator, self).__init__()
    self.proj = nn.Linear(d_model, vocab)
  
  def forward(self,x):
    return log_softmax(self.proj(x),dim=-1)

In [ ]:
def clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
class Encoder(nn.Module):
  def __init__(self,layer,N):
    super(Encoder, self).__init__()
    self.layers = clones(layer,N)
    self.norm = LayerNorm(layer.size)
  
  def forward(self,x,mask):
    for layer in self.layers:
      x = layer(x,mask)
    return self.norm(x)

In [ ]:
class LayerNorm(nn.Module):
  def __init__(self,features,eps=1e-6):
    super(LayerNorm,self).__init__()
    self.a_2 = nn.Parameter(torch.ones(features))
    self.b_2 = nn.Parameter(torch.zeros(features))
    self.eps = eps

  def forward(self,x):
    mean = x.mean(-1,keepdim=True)
    std = x.std(-1,keepdim=True)
    return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
class SublayerConnection(nn.Module):
  def __init__(self,size,dropout):
    super(SublayerConnection,self).__init__()
    self.norm = LayerNorm(size)
    self.dropout = nn.Dropout(dropout)
  
  def forward(self,x,sublayer):
    return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self,size,self_attn,feed_forward,dropout):
    super(EncoderLayer,self).__init__()
    self.self_attn = self_attn
    self.feed_forward = feed_forward
    self.sublayer = clones(SublayerConnection(size,dropout),2)
    self.size = size
  
  def forward(self,x,mask):
    x = self.sublayer[0](x,lambda x: self.self_attn(x,x,x,mask))
    return self.sublayer[1](x,self.feed_forward)

In [ ]:
class Decoder(nn.Module):
  def __init__(self,layer,N):
    super(Decoder,self).__init__()
    self.layers = clones(layer,N)
    self.norm = LayerNorm(layer.size)

  def forward(self,x,memory,src_mask,tgt_mask):
    for layer in self.layers:
      x = layer(x,memory,src_mask,tgt_mask)
    return self.norm(x)

In [ ]:
class DecoderLayer(nn.Module):
  def __init__(self,size,self_attn,src_attn,feed_forward,dropout):
    super(DecoderLayer,self).__init__()
    self.size = size
    self.self_attn = self_attn
    self.src_attn = src_attn
    self.feed_forward = feed_forward
    self.sublayer = clones(SublayerConnection(size,dropout),3)

  def forward(self,x,memory,src_mask,tgt_mask):
    m = memory
    x = self.sublayer[0](x,lambda x: self.self_attn(x,x,x,tgt_mask))
    x = self.sublayer[1](x,lambda x: self.src_attn(x,m,m,src_mask))
    return self.sublayer[2](x,self.feed_forward)

In [ ]:
def subsequent_mask(size):
  attn_shape = (1,size,size)
  subsequent_mask = torch.triu(torch.ones(attn_shape),diagonal=1).type(torch.uint8)
  return subsequent_mask == 0

In [ ]:
def attention(query,key,value,mask=None,dropout=None):
  d_k = query.size(-1)
  scores = torch.matmul(query,key.transpose(-2,-1)) / math.sqrt(d_k)
  if mask is not None:
    scores = scores.masked_fill(mask == 0, -1e9)
  p_attn = scores.softmax(dim=-1)
  if dropout is not None:
    p_attn = dropout(p_attn)
  return torch.matmul(p_attn,value),p_attn

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self,h,d_model,dropout=0.1):
    super(MultiHeadAttention,self).__init__()
    assert d_model % h == 0
    #we assume d_v always equals d_k
    self.d_k = d_model // h
    self.h = h
    self.linears = clones(nn.Linear(d_model,d_model),4)
    self.attn = None
    self.dropout = nn.Dropout(p=dropout)

  def forward(self,query,key,value,mask=None):
    if mask is not None:
      mask = mask.unsqueeze(1)
    nbatches = query.size(0)
    
    query,key,value = [
        lin(x).view(nbatches,-1,self.h,self.d_k).transpose(1,2)
        for lin,x in zip(self.linears,(query,key,value))
    ]
  
    x, self.attn = attention(
        query,key,value,mask=mask,dropout=self.dropout
    )

    x =  (
        x.transpose(1,2).contiguous().view(nbatches,-1,self.h * self.d_k)
    )

    del query
    del key
    del value
    return self.linears[-1](x)

In [ ]:
class PositionWiseFeedForward(nn.Module):
  def __init__(self,d_model,d_ff,dropout=0.1):
    super(PositionWiseFeedForward,self).__init__()
    self.w_1 = nn.Linear(d_model, d_ff)
    self.w_2 = nn.Linear(d_ff,d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    return self.w_2(self.dropout(self.w_1(x).relu()))

In [ ]:
class Embeddings(nn.Module):
  def __init__(self, d_model,vocab):
    super(Embeddings,self).__init__()
    self.lut = nn.Embedding(vocab,d_model)
    self.d_model = d_model

  def forward(self,x):
    return self.lut(x) * math.sqrt(self.d_model)

In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self,d_model,dropout,max_len=5000):
    super(PositionalEncoding,self).__init__()
    self.dropout = nn.Dropout(p=dropout)

    pe = torch.zeros(max_len,d_model)
    position = torch.arange(0,max_len).unsqueeze(1)
    div_term = torch.exp(
        torch.arange(0,d_model,2) * (-math.log(10000.0) / d_model)
    )
    pe[:,0::2] = torch.sin(position * div_term)
    pe[:,1::2] = torch.cos(position * div_term)
    pe = pe.unsqueeze(0)
    self.register_buffer("pe",pe)
  
  def forward(self,x):
    x = x + self.pe[:, :x.size(1)].requires_grad_(False)
    return self.dropout(x)

In [ ]:
def make_model(
    src_vocab, tgt_vocab, N=6, d_model=512,d_ff=2048,h=8,dropout=0.1
):
  c = copy.deepcopy
  attn = MultiHeadAttention(h,d_model)
  ff = PositionWiseFeedForward(d_model,d_ff,dropout)
  position = PositionalEncoding(d_model,dropout)
  model = EncoderDecoder(
      Encoder(EncoderLayer(d_model,c(attn),c(ff),dropout),N),
      Decoder(DecoderLayer(d_model,c(attn),c(attn),c(ff),dropout),N),
      nn.Sequential(Embeddings(d_model,src_vocab),c(position)),
      nn.Sequential(Embeddings(d_model,tgt_vocab),c(position)),
      Generator(d_model,tgt_vocab),
  )

  #Weights initialization
  for p in model.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform_(p)

  return model

In [ ]:
def inference_test():
  test_model = make_model(11,11,2)
  test_model.eval()
  src = torch.LongTensor([[1,2,3,4,5,6,7,8,9,10]])
  src_mask = torch.ones(1,1,10)

  memory = test_model.encode(src,src_mask)
  ys = torch.zeros(1,1).type_as(src)

  for i in range(10):
    out = test_model.decode(
        memory,src_mask,ys,subsequent_mask(ys.size(1)).type_as(src.data)
    )
    prob = test_model.generator(out[:,-1])
    _,next_word = torch.max(prob,dim=1)
    next_word = next_word.data[0]
    ys = torch.cat(
        [ys,torch.empty(1,1).type_as(src.data).fill_(next_word)],dim=1
    )
  
  print("Example Untrained Model Prediction: ",ys)


def run_tests():
  for _ in range(10):
    inference_test()


show_example(run_tests)

Example Untrained Model Prediction:  tensor([[ 0,  3,  8,  4, 10, 10, 10, 10, 10, 10, 10]])
Example Untrained Model Prediction:  tensor([[0, 8, 7, 9, 7, 9, 7, 9, 7, 9, 7]])
Example Untrained Model Prediction:  tensor([[ 0,  2, 10,  1,  1,  1,  1,  1,  1,  1,  1]])
Example Untrained Model Prediction:  tensor([[0, 3, 2, 1, 3, 2, 3, 2, 3, 2, 3]])
Example Untrained Model Prediction:  tensor([[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]])
Example Untrained Model Prediction:  tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
Example Untrained Model Prediction:  tensor([[0, 1, 7, 9, 0, 1, 1, 1, 1, 1, 7]])
Example Untrained Model Prediction:  tensor([[0, 7, 1, 7, 1, 7, 1, 1, 1, 1, 1]])
Example Untrained Model Prediction:  tensor([[ 0, 10,  9,  9,  2,  7,  4,  4,  9, 10,  5]])
Example Untrained Model Prediction:  tensor([[0, 3, 0, 2, 5, 7, 3, 0, 2, 5, 4]])


In [ ]:
class Batch:
  def __init__(self,src,tgt=None,pad=2):
    self.src = src
    self.src_mask = (src != pad).unsqueeze(-2)
    if tgt is not None:
      self.tgt = tgt[:,:-1]
      self.tgt_y = tgt[:,1:]
      self.tgt_mask = self.make_std_mask(self.tgt, pad)
      self.ntokens = (self.tgt_y != pad).data.sum()

  @staticmethod
  def make_std_mask(tgt,pad):
    tgt_mask = (tgt != pad).unsqueeze(-2)
    tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data)
    return tgt_mask

In [ ]:
class TrainState:
  step: int = 0
  accum_step : int = 0
  samples : int = 0
  tokens : int = 0

In [ ]:
def run_epoch(
    data_iter,model,loss_compute,optimizer,
    scheduler,mode="train",accum_iter=1,train_state=TrainState(),
):
  start = time.time()
  total_tokens = 0
  total_loss = 0
  tokens = 0
  n_accum = 0
  for i,batch in enumerate(data_iter):
    out = model.forward(batch.src,batch.tgt,batch.src_mask,batch.tgt_mask)
    loss,loss_node = loss_compute(out,batch.tgt_y,batch.ntokens)
    if mode == "train" or mode == "train+log":
      loss_node.backward()
      train_state.step += 1
      train_state.samples += batch.src.shape[0]
      train_state.tokens += batch.ntokens
      if i % accum_iter == 0:
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        n_accum += 1
        train_state.accum_step += 1
      scheduler.step()

    total_loss += loss
    total_tokens += batch.ntokens
    tokens += batch.ntokens
    if i % 40 == 1 and (mode == "train" or mode == "train+log"):
      lr = optimizer.param_groups[0]["lr"]
      elapsed = time.time() - start
      print(
          ("Epoch Step: %6d | Accumulation Step: %3d | Loss: %6.2f " +
          "|Tokens / Sec: %7.1f | Learning Rate: %6.1e"
          ) % (i, n_accum, loss / batch.ntokens, tokens / elapsed, lr)
      )
      start = time.time()
      tokens = 0
    del loss
    del loss_node
  return total_loss / total_tokens, train_state

In [ ]:
def rate(step,model_size,factor,warmup):
  if step == 0:
    step = 1
  return factor * (
      model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5))
  )

In [ ]:
class LabelSmoothing(nn.Module):
  def __init__(self,size,padding_idx,smoothing=0.0):
    super(LabelSmoothing,self).__init__()
    self.criterion = nn.KLDivLoss(reduction="sum")
    self.padding_idx = padding_idx
    self.smoothing = smoothing
    self.confidence = 1.0 - smoothing
    self.size = size
    self.true_dist = None
  
  def forward(self,x,target):
    assert x.size(1) == self.size
    true_dist = x.data.clone()
    true_dist.fill_(self.smoothing / (self.size - 2))
    true_dist.scatter_(1,target.data.unsqueeze(1),self.confidence)
    true_dist[:,self.padding_idx] = 0
    mask = torch.nonzero(target.data == self.padding_idx)
    if mask.dim() > 0:
      true_dist.index_fill_(0,mask.squeeze(),0.0)
    self.true_dist = true_dist
    return self.criterion(x,true_dist.clone().detach()) 

In [ ]:
class SimpleLossCompute:
  def __init__(self,generator,criterion):
    self.generator = generator
    self.criterion = criterion

  def __call__(self,x,y,norm):
    x = self.generator(x)
    sloss = self.criterion(x.contiguous().view(-1,x.size(-1)),y.contiguous().view(-1)) / norm

    return sloss.data * norm, sloss

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.zeros(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len - 1):
        out = model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.zeros(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )
    return ys

In [ ]:
def data_gen(V, batch_size, nbatches):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.randint(1, V, size=(batch_size, 10))
        data[:, 0] = 1
        src = data.requires_grad_(False).clone().detach()
        tgt = data.requires_grad_(False).clone().detach()
        yield Batch(src, tgt, 0)

In [ ]:
def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)

In [ ]:
from tqdm import tqdm

In [ ]:
def example_simple_model():
    V = 11
    criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
    model = make_model(V, V, N=2)

    optimizer = torch.optim.Adam(
        model.parameters(), lr=0.5, betas=(0.9, 0.98), eps=1e-9
    )
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(
            step, model_size=model.src_embed[0].d_model, factor=1.0, warmup=400
        ),
    )

    batch_size = 80
    for epoch in tqdm(range(120)):
        model.train()
        run_epoch(
            data_gen(V, batch_size, 20),
            model,
            SimpleLossCompute(model.generator, criterion),
            optimizer,
            lr_scheduler,
            mode="train",
        )
        model.eval()
        run_epoch(
            data_gen(V, batch_size, 5),
            model,
            SimpleLossCompute(model.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode="eval",
        )[0]

    model.eval()
    src = torch.LongTensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
    max_len = src.shape[1]
    src_mask = torch.ones(1, 1, max_len)
    print(greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0))


execute_example(example_simple_model)

  0%|          | 0/120 [00:00<?, ?it/s]

Epoch Step:      1 | Accumulation Step:   2 | Loss:   3.10 |Tokens / Sec:   439.2 | Learning Rate: 5.5e-06


  1%|          | 1/120 [00:34<1:08:31, 34.55s/it]

Epoch Step:      1 | Accumulation Step:   2 | Loss:   2.14 |Tokens / Sec:   466.6 | Learning Rate: 6.1e-05


  2%|▏         | 2/120 [01:07<1:05:46, 33.44s/it]

...

Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.00 |Tokens / Sec:   256.9 | Learning Rate: 4.5e-04


 99%|█████████▉| 119/120 [1:28:04<00:58, 58.58s/it]

Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.00 |Tokens / Sec:   257.2 | Learning Rate: 4.5e-04


100%|██████████| 120/120 [1:29:02<00:00, 44.52s/it]

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
